# Optimización de Hiperparámetros en Deep Learning

Este notebook presenta ejemplos prácticos de las principales herramientas para optimización de hiperparámetros mencionadas en el documento de evaluación y optimización de modelos de Machine Learning.

## ¿Qué son los hiperparámetros?

Los hiperparámetros son valores que **nosotros decidimos antes del entrenamiento** y que no son aprendidos por el modelo. Ejemplos incluyen:

- Número de capas
- Número de neuronas por capa
- Tasa de aprendizaje (learning rate)
- Tasa de dropout
- Tipo de optimizador

Encontrar la combinación óptima de estos valores es crucial para obtener el mejor rendimiento del modelo.

---
## 1. Keras Tuner

### ¿Qué es?
**Keras Tuner** es la solución oficial de TensorFlow/Keras para la búsqueda automatizada de hiperparámetros. Fue desarrollada por el equipo de Keras y se integra perfectamente con el ecosistema de TensorFlow.

### Características principales:
- **Integración nativa**: Diseñada específicamente para modelos Keras/TensorFlow
- **Múltiples algoritmos**: Soporta búsqueda aleatoria, bayesiana e Hyperband
- **Fácil de usar**: API intuitiva y bien documentada
- **Reanudable**: Puede continuar búsquedas interrumpidas

### Algoritmos disponibles:
- `RandomSearch`: Prueba combinaciones aleatorias
- `BayesianOptimization`: Usa modelos probabilísticos para guiar la búsqueda
- `Hyperband`: Asigna recursos adaptativamente, descartando configuraciones pobres

### Cuándo usarlo:
- Cuando trabajas exclusivamente con TensorFlow/Keras
- Para proyectos que requieren una solución simple y bien integrada
- Cuando necesitas soporte oficial y documentación extensa

In [1]:
# Instalación de Keras Tuner
!pip install keras-tuner -q


[notice] A new release of pip is available: 23.0.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importaciones necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras Tuner version: {kt.__version__}")

TensorFlow version: 2.20.0
Keras Tuner version: 1.4.8


In [3]:
# Definir el modelo con hiperparámetros buscables
def build_model(hp):
    """
    Construye un modelo con hiperparámetros definidos por el tuner.
    
    hp.Int(): Define un hiperparámetro entero
    hp.Float(): Define un hiperparámetro de punto flotante
    hp.Choice(): Define un hiperparámetro categórico
    """
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    # Buscar el número óptimo de capas (entre 1 y 3)
    for i in range(hp.Int('num_layers', 1, 3)):
        # Buscar el número óptimo de neuronas (32 a 512, en pasos de 32)
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
            activation='relu'
        ))
        # Buscar el dropout óptimo (0.0 a 0.5)
        model.add(layers.Dropout(hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1)))
    
    model.add(layers.Dense(10, activation='softmax'))
    
    # Buscar la tasa de aprendizaje óptima (escala logarítmica)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [4]:
# Cargar y preparar datos MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(f"Datos de entrenamiento: {x_train.shape}")
print(f"Datos de prueba: {x_test.shape}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Datos de entrenamiento: (60000, 28, 28)
Datos de prueba: (10000, 28, 28)


In [5]:
# Configurar el tuner con búsqueda bayesiana
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',      # Métrica a optimizar
    max_trials=20,                  # Número máximo de configuraciones a probar
    directory='keras_tuner_results',
    project_name='mnist_optimization'
)

# Ver resumen del espacio de búsqueda
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


e:\github\pia\venv310\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
# Ejecutar la búsqueda de hiperparámetros
tuner.search(
    x_train, y_train,
    epochs=10,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

Trial 6 Complete [00h 01m 28s]
val_accuracy: 0.9745833277702332

Best val_accuracy So Far: 0.9794166684150696
Total elapsed time: 00h 11m 58s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
1                 |3                 |num_layers
320               |320               |units_0
0.3               |0.2               |dropout_0
0.00013759        |0.00020635        |learning_rate
224               |480               |units_1
0                 |0.3               |dropout_1
352               |32                |units_2
0.3               |0                 |dropout_2

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.8456 - loss: 0.5662 - val_accuracy: 0.9270 - val_loss: 0.2699
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9223 - loss: 0.2735 - val_accuracy: 0.9457 - val_loss: 0.2056
Epoch 3/10
 431/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9392 - loss: 0.2174

KeyboardInterrupt: 

In [ ]:
# Obtener los mejores hiperparámetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("=" * 50)
print("MEJORES HIPERPARÁMETROS ENCONTRADOS")
print("=" * 50)
print(f"Número de capas: {best_hps.get('num_layers')}")
print(f"Learning rate: {best_hps.get('learning_rate'):.6f}")

for i in range(best_hps.get('num_layers')):
    print(f"Capa {i+1} - Unidades: {best_hps.get(f'units_{i}')}, Dropout: {best_hps.get(f'dropout_{i}'):.1f}")

In [ ]:
# Entrenar el modelo final con los mejores hiperparámetros
best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(
    x_train, y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

# Evaluar en datos de prueba
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"\nPrecisión en test: {test_acc:.4f}")

---
## 2. Optuna

### ¿Qué es?
**Optuna** es un framework moderno y eficiente para optimización de hiperparámetros. Es agnóstico al framework de deep learning, lo que significa que funciona con TensorFlow, PyTorch, scikit-learn y cualquier otra biblioteca.

### Características principales:
- **Poda inteligente (Pruning)**: Detiene automáticamente entrenamientos que no son prometedores
- **Visualizaciones integradas**: Gráficos interactivos para analizar resultados
- **Define-by-run**: API flexible que permite definir el espacio de búsqueda dinámicamente
- **Paralelización**: Soporta búsquedas distribuidas
- **Persistencia**: Guarda estudios en bases de datos (SQLite, MySQL, PostgreSQL)

### Conceptos clave:
- **Study**: Un proceso de optimización completo
- **Trial**: Una ejecución individual con una configuración específica
- **Objective**: La función que queremos minimizar o maximizar

### Cuándo usarlo:
- Cuando necesitas flexibilidad para usar diferentes frameworks
- Para proyectos que requieren poda inteligente de trials
- Cuando quieres visualizaciones detalladas del proceso de optimización

In [ ]:
# Instalación de Optuna
!pip install optuna plotly -q

In [ ]:
# Importaciones necesarias
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Silenciar logs de TensorFlow
tf.get_logger().setLevel('ERROR')

print(f"Optuna version: {optuna.__version__}")

In [ ]:
# Cargar y preparar datos
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Separar datos de validación
x_val, y_val = x_train[:10000], y_train[:10000]
x_train, y_train = x_train[10000:], y_train[10000:]

print(f"Entrenamiento: {x_train.shape}")
print(f"Validación: {x_val.shape}")
print(f"Test: {x_test.shape}")

In [ ]:
# Definir la función objetivo
def objective(trial):
    """
    Función objetivo que Optuna intentará maximizar.
    
    trial.suggest_int(): Sugiere un valor entero
    trial.suggest_float(): Sugiere un valor flotante
    trial.suggest_categorical(): Sugiere un valor de una lista
    """
    # Sugerir hiperparámetros
    n_layers = trial.suggest_int('n_layers', 1, 4)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    # Construir modelo
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    for i in range(n_layers):
        n_units = trial.suggest_int(f'n_units_l{i}', 32, 256)
        dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.1, 0.5)
        model.add(layers.Dense(n_units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(10, activation='softmax'))
    
    # Configurar optimizador
    if optimizer_name == 'Adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callback para poda (pruning)
    class OptunaCallback(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            # Reportar valor intermedio
            trial.report(logs['val_accuracy'], epoch)
            # Verificar si debemos podar este trial
            if trial.should_prune():
                raise optuna.TrialPruned()
    
    # Entrenar
    history = model.fit(
        x_train, y_train,
        epochs=20,
        batch_size=batch_size,
        validation_data=(x_val, y_val),
        callbacks=[OptunaCallback()],
        verbose=0
    )
    
    return history.history['val_accuracy'][-1]

In [ ]:
# Crear y ejecutar el estudio
study = optuna.create_study(
    direction='maximize',  # Queremos maximizar la accuracy
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5,   # Trials antes de empezar a podar
        n_warmup_steps=5      # Epochs antes de podar un trial
    ),
    study_name='mnist_optimization'
)

# Ejecutar optimización
study.optimize(
    objective,
    n_trials=30,
    show_progress_bar=True
)

In [ ]:
# Ver resultados
print("=" * 50)
print("RESULTADOS DE OPTUNA")
print("=" * 50)
print(f"Número de trials completados: {len(study.trials)}")
print(f"Mejor valor de accuracy: {study.best_trial.value:.4f}")
print(f"\nMejores hiperparámetros:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

In [ ]:
# Visualización: Historia de optimización
fig = plot_optimization_history(study)
fig.show()

In [ ]:
# Visualización: Importancia de parámetros
fig = plot_param_importances(study)
fig.show()

In [ ]:
# Visualización: Coordenadas paralelas
fig = plot_parallel_coordinate(study)
fig.show()

---
## 3. Ray Tune

### ¿Qué es?
**Ray Tune** es una biblioteca para búsqueda de hiperparámetros distribuida y escalable. Es parte del ecosistema Ray, diseñado para computación distribuida de alto rendimiento.

### Características principales:
- **Escalabilidad**: Ejecuta búsquedas en clusters de múltiples máquinas
- **Schedulers avanzados**: ASHA, PBT (Population Based Training), etc.
- **Integración**: Compatible con Optuna, HyperOpt, y otros algoritmos
- **Tolerancia a fallos**: Recuperación automática de experimentos fallidos
- **Soporte multi-GPU**: Distribución eficiente de recursos

### Schedulers disponibles:
- **ASHAScheduler**: Detención temprana agresiva pero eficiente
- **PopulationBasedTraining**: Evoluciona hiperparámetros durante el entrenamiento
- **MedianStoppingRule**: Detiene trials por debajo de la mediana

### Cuándo usarlo:
- Para búsquedas a gran escala con múltiples GPUs o máquinas
- Cuando necesitas tolerancia a fallos en experimentos largos
- Para proyectos de producción que requieren escalabilidad

In [ ]:
# Instalación de Ray Tune
!pip install "ray[tune]" -q

In [ ]:
# Importaciones necesarias
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

print(f"Ray version: {ray.__version__}")

In [ ]:
# Inicializar Ray
ray.init(ignore_reinit_error=True)

In [ ]:
# Función para cargar datos
def load_data():
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    return (x_train, y_train), (x_test, y_test)

In [ ]:
# Función de entrenamiento para Ray Tune
def train_mnist(config):
    """
    Función de entrenamiento que recibe configuración de Ray Tune.
    
    config: diccionario con los hiperparámetros a probar
    """
    (x_train, y_train), (x_test, y_test) = load_data()
    
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(config["hidden_units_1"], activation='relu'),
        layers.Dropout(config["dropout_1"]),
        layers.Dense(config["hidden_units_2"], activation='relu'),
        layers.Dropout(config["dropout_2"]),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config["learning_rate"]),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Callback para reportar métricas a Ray Tune
    class TuneReportCallback(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            tune.report(
                mean_accuracy=logs['val_accuracy'],
                mean_loss=logs['val_loss']
            )
    
    model.fit(
        x_train, y_train,
        epochs=20,
        validation_data=(x_test, y_test),
        batch_size=config["batch_size"],
        callbacks=[TuneReportCallback()],
        verbose=0
    )

In [ ]:
# Configurar el espacio de búsqueda
search_space = {
    "hidden_units_1": tune.choice([64, 128, 256, 512]),
    "hidden_units_2": tune.choice([32, 64, 128, 256]),
    "dropout_1": tune.uniform(0.1, 0.5),
    "dropout_2": tune.uniform(0.1, 0.5),
    "learning_rate": tune.loguniform(1e-5, 1e-2),
    "batch_size": tune.choice([32, 64, 128])
}

In [ ]:
# Configurar scheduler ASHA para detención temprana
scheduler = ASHAScheduler(
    metric="mean_accuracy",
    mode="max",
    max_t=20,           # Máximo de epochs
    grace_period=5,     # Epochs mínimos antes de podar
    reduction_factor=2  # Factor de reducción
)

# Usar Optuna como algoritmo de búsqueda
search_alg = OptunaSearch(
    metric="mean_accuracy",
    mode="max"
)

In [ ]:
# Ejecutar la búsqueda
analysis = tune.run(
    train_mnist,
    config=search_space,
    num_samples=20,           # Número de configuraciones a probar
    scheduler=scheduler,
    search_alg=search_alg,
    resources_per_trial={"cpu": 2, "gpu": 0},  # Recursos por trial
    verbose=1
)

In [ ]:
# Obtener mejores resultados
best_config = analysis.get_best_config(metric="mean_accuracy", mode="max")

print("=" * 50)
print("RESULTADOS DE RAY TUNE")
print("=" * 50)
print("Mejor configuración encontrada:")
for key, value in best_config.items():
    print(f"  {key}: {value}")

In [ ]:
# Ver DataFrame con todos los resultados
df = analysis.results_df
print("\nTop 5 configuraciones:")
df.sort_values("mean_accuracy", ascending=False).head()

In [ ]:
# Apagar Ray
ray.shutdown()

---
## 4. TensorBoard HParams

### ¿Qué es?
**TensorBoard HParams** es un plugin de TensorBoard que permite visualizar y comparar de forma interactiva los resultados de múltiples experimentos con diferentes hiperparámetros.

### Características principales:
- **Visualización interactiva**: Tablas, gráficos de coordenadas paralelas y scatter plots
- **Filtrado dinámico**: Filtra experimentos por rangos de hiperparámetros
- **Comparación lado a lado**: Compara curvas de entrenamiento de diferentes configuraciones
- **Integración con TensorBoard**: Usa la misma interfaz que ya conoces

### Componentes clave:
- **HParam**: Define un hiperparámetro y su dominio
- **Metric**: Define las métricas a registrar
- **hparams_config**: Configura el experimento

### Cuándo usarlo:
- Para visualizar y analizar resultados de búsquedas manuales o automatizadas
- Cuando necesitas una interfaz visual para explorar el espacio de hiperparámetros
- Para presentar resultados de experimentación de forma clara

In [ ]:
# Importaciones necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorboard.plugins.hparams import api as hp
import datetime
import shutil
import os

In [ ]:
# Definir hiperparámetros a explorar
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128, 256]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.3, 0.5]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-2, 1e-3, 1e-4]))

# Métricas a registrar
METRIC_ACCURACY = 'accuracy'

In [ ]:
# Configurar el directorio de logs
log_dir = "logs/hparam_tuning/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Limpiar directorio si existe
if os.path.exists("logs/hparam_tuning"):
    shutil.rmtree("logs/hparam_tuning")

# Configurar HParams en TensorBoard
with tf.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_LEARNING_RATE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [ ]:
# Cargar datos
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# Función de entrenamiento
def train_test_model(hparams, run_dir):
    """Entrena un modelo con los hiperparámetros dados y retorna la accuracy."""
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
        layers.Dropout(hparams[HP_DROPOUT]),
        layers.Dense(10, activation='softmax'),
    ])
    
    # Configurar optimizador
    if hparams[HP_OPTIMIZER] == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE])
    else:
        optimizer = keras.optimizers.SGD(learning_rate=hparams[HP_LEARNING_RATE], momentum=0.9)
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    
    # Entrenar con callback de TensorBoard
    model.fit(
        x_train, y_train,
        epochs=5,  # Reducido para demostración
        validation_split=0.2,
        callbacks=[keras.callbacks.TensorBoard(run_dir, histogram_freq=1)],
        verbose=0
    )
    
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

In [ ]:
# Ejecutar experimentos con todas las combinaciones
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            for lr in HP_LEARNING_RATE.domain.values:
                hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_DROPOUT: dropout_rate,
                    HP_OPTIMIZER: optimizer,
                    HP_LEARNING_RATE: lr,
                }
                
                run_name = f"run-{session_num}"
                run_dir = log_dir + "/" + run_name
                
                print(f'--- Trial {session_num + 1}: units={num_units}, dropout={dropout_rate}, '
                      f'opt={optimizer}, lr={lr}')
                
                # Entrenar y obtener accuracy
                accuracy = train_test_model(hparams, run_dir)
                print(f'    Accuracy: {accuracy:.4f}')
                
                # Registrar hiperparámetros y métrica en TensorBoard
                with tf.summary.create_file_writer(run_dir).as_default():
                    hp.hparams(hparams)
                    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
                
                session_num += 1

print(f"\nTotal de experimentos: {session_num}")

In [ ]:
# Cargar extensión de TensorBoard
%load_ext tensorboard

In [ ]:
# Visualizar en TensorBoard
# Navega a la pestaña "HPARAMS" para ver las visualizaciones interactivas
%tensorboard --logdir logs/hparam_tuning

---
## Resumen Comparativo

| Herramienta | Ventaja Principal | Mejor Caso de Uso | Curva de Aprendizaje |
|-------------|-------------------|-------------------|----------------------|
| **Keras Tuner** | Integración nativa con Keras | Proyectos TensorFlow/Keras | Baja |
| **Optuna** | Poda inteligente, visualizaciones | Cualquier framework, flexibilidad | Media |
| **Ray Tune** | Escalabilidad distribuida | Clusters, múltiples GPUs | Alta |
| **TensorBoard HParams** | Visualización interactiva | Análisis de resultados | Baja |

### Recomendaciones:

1. **Para principiantes**: Comienza con **Keras Tuner** si usas TensorFlow/Keras
2. **Para proyectos medianos**: **Optuna** ofrece el mejor balance entre funcionalidad y facilidad
3. **Para producción a escala**: **Ray Tune** cuando necesites distribuir la búsqueda
4. **Para análisis visual**: Combina cualquier herramienta con **TensorBoard HParams**

### Consejo del documento original:

> *"Aunque estas herramientas automáticas son útiles, es fundamental entender qué hace cada hiperparámetro para poder interpretar los resultados y tomar decisiones informadas sobre la arquitectura de tu modelo."*